In [1]:
#-----------------------------------------------------------------------
# SIMULATION PARAMETERS FOR THE PIC-CODE SMILEI -- REFLECTIVE WALL CASE
# ----------------------------------------------------------------------

import math as m
import numpy as np
#
q  = 4.80326e-10       #statCoulomb
c  = 3.*1e10           #cm/s
me = 9.10938356*1e-28  #g
mi = 1836  #me/me = 1, mi/me = 1836


cost = 1.380649e-23 #/ 8.617333262e-5
## INPUT ##



# Eon plasma frequency in 1/s
v0 = 1.5e7 #velocity in cm/s --> 300 km/s 
v0_c = v0/c

## ------- PISTON PLASMA -------

TeP_eV = 45 # temperature of electrons in eV
TiP_eV = 45 # temperature of ions in eV 
TeP_mec2 = TeP_eV/(511*1e3) # temperature nondimensionalization
TiP_mec2 = TiP_eV/(511*1e3) # temperature nondimensionalization

ZP = 6  #non fully ionized N2 -- see FLASH sim.

cost = 1.380649e-23 #/ 8.617333262e-5

#ne = 10 mbar / (kb* 290 K)  = ion density -->
#eon density for reference= ion * ZP * density jump condition (max 4)
p = 10 #mbar
p = p * 100 #Pascal
T_chamber = 290 #K
n0 = p/cost/T_chamber/1e6 * ZP * 4 
print( 'reference density: ' + str(n0))


neP = n0 #This actually has a profile, varies from 1e17 to 1e20 cm-3 
niP = neP/ZP

neP_n0 = neP/n0 #density nondimensionalization
niP_n0 = niP/n0 #density nondimensionalization

miP = mi*14 




#--------Parameters for Input file----------------------------------------------------------------------
wpe = m.sqrt(4.*m.pi*n0*q**2/me)            # Eon plasma frequency in 1/s
c_wpe = c/wpe                               # Electron skin depth is reference length for Smilei), in cm.
inv_ld2 = (1/7.43e2)**2*(n0/TeP_eV)*(1+ZP)  # 1/lambda_D^2
deb_len = 1/np.sqrt(inv_ld2)                # Debye length

#------------- Case 1 -----------------------------------------------------------------------------------



cost = 5  ## Should be < 3
dx = cost*deb_len/c_wpe


#--------------------------------------------------------------------------------------------------------

cell_per_patch = 6  #min 6
num_patches = 2**16 #power of 2 (prioritize large number of patches?)

#--------------------------------------------------------------------------------------------------------

Nx= num_patches*cell_per_patch 
Lx = dx*Nx 

#Now, one should compare Lx with Lx_rough
print('Simulation box length (Nx*dx) :' +str(Lx*c_wpe) + ' cm')
print('')


Nppc = 300 #?
#---------------------------------------------------------------------------------------------------------

# Normalization time in ns
dt = 0.95*dx  
Tsim_ns = 4 #Simulation time in ns
t_sim = int(Tsim_ns*wpe/1e9) #nondimensionalization 
Nt = t_sim/dt
push_time = 600 #ns hopefully, a conservative value.

#---------------------------------------------------------------------------------------------------------


Expected_time_h = 2*Nppc*Nt*Nx*push_time/1e9/3600


print('Nppc =' +str( Nppc))
#print(5/5.33)
print('Expected time in hours: ' +str(Expected_time_h))
#print(Nt)

reference density: 5.994182496033037e+18
Simulation box length (Nx*dx) :1.5128025762197312 cm

Nppc =300
Expected time in hours: 645516.7621756898


In [2]:
import numpy as np
import os
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.interpolate import CubicSpline
from scipy.interpolate import UnivariateSpline
from scipy.special import erf as erf

# ION ION COLLISION
e = 4.8e-10
cost = 1.380649e-16/ 8.617333262e-5 #erg/K * K/eV --> cost* eV = erg = cm^2*g/s^2 

me  = 9.10938356e-28 #g
c= 3e10 #cm/s

A = 14 #N2
Z = ZP #charge state
v = v0 #cm/s
mi = 1836*A*me
Ti = TiP_eV
Te = TeP_eV
ne = n0/4
ni = ne/ZP
v_rel = v*2

gamma = 5/3
cs_ion = 9.79e5*(gamma*Z*Te/A)**0.5
Mach = v_rel/cs_ion


mu = A
beta = v_rel/c
#counterstreaming ions with hot background electrons
cLog = 43 - np.log(Z**2*(2*mu)/(mu**2)/beta**2*(ne/Te)**(0.5))

x_ii = mi*v_rel**2/2/cost/Ti
    
psi_ii = erf(np.sqrt(x_ii)) -2./np.sqrt(np.pi)*np.exp(-x_ii)*np.sqrt(x_ii)
eps = 0.5*mi*v_rel**2/cost
f_ii_slow = 6.8e-8*A**(-0.5)*2*Te**(-1.5) *ni*Z**4*cLog
f_ii_fast = 9.0e-8 * (2/A)*A**0.5/eps**1.5 *ni*Z**4*cLog

nu0 = 4*np.pi*ni*Z**2*Z**2*cLog*e**4/mi**2/v_rel**3
nu_ii = nu0*psi_ii*(2)


print('Mach: ' + str(Mach))
print('Coulomb logarithm: ' + str(cLog))
print('Slow/Fast regime? ' +str(x_ii))
print('')
print('Collision frequency NRL (fast regime): ' + str(f_ii_fast/1e12) + ' E12 1/s' )
print('')
print('Collision frequency NRL (slow regime): ' + str(f_ii_slow/1e12) + ' E12 1/s' )
print('')
print('Collision frequency (correct): ' + str(nu_ii/1e12) + ' E12 1/s' )
print('')
print('Collisional time: '+ str(1/nu_ii*1e12) + ' ps')

Mach: 5.405007785627356
Coulomb logarithm: 8.524698533692153
Slow/Fast regime? 146.14343390948198

Collision frequency NRL (fast regime): 0.00024889920282078053 E12 1/s

Collision frequency NRL (slow regime): 0.3322453186485414 E12 1/s

Collision frequency (correct): 0.0002486934761248366 E12 1/s

Collisional time: 4021.014204240847 ps
